# Optimización Cartera de Préstamos a Titulizar

In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
from pyspark.sql import functions as F, DataFrame
import datetime as dt
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.window import Window
import pyspark.sql.types as t
from decimal import Decimal
from pyspark.sql.functions import regexp_replace
import itertools

In [2]:
from dataproc_sdk.dataproc_sdk_datiopysparksession.datiopysparksession import DatioPysparkSession
datioSparkSession = DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_datiopysparksession import datiopysparksession
dataproc = datiopysparksession.DatioPysparkSession().get_or_create()

from dataproc_sdk.dataproc_sdk_schema.datioschema import DatioSchema

## Instalamos librerías

In [3]:
path_csv = '/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/modelo_optimizacion/SRT_Model_Optimizador-GPT.csv'
path_csv ='/data/sandboxes/dslb/data/Joystick/TITULIZACIONES/modelo_optimizacion'

In [5]:
# SRT_Model_Optimizador-GPT.csv

## Leemos CSV con la Cartera de Préstamos

In [9]:
df = spark.read.option('header','True').option('delimiter',',').csv(path_csv)
df.show(5,False)

+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
|Nº Expediente|Tramo|País    |Clase Producto Financiero|Divisa Base|Importe del Expediente|CR_Porcentaje|
+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
|108870       |2    |PORTUGAL|Amortizing               |EUR        |28062.51              |4.0          |
|108949       |2    |FRANCIA |Amortizing               |EUR        |1335001.71            |5.6          |
|108949       |3    |FRANCIA |Amortizing               |EUR        |1304587.74            |5.6          |
|108949       |4    |FRANCIA |Amortizing               |EUR        |1326868.02            |5.6          |
|108949       |5    |FRANCIA |Amortizing               |EUR        |1618368.82            |5.6          |
+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
only showing top 5 rows



In [14]:
df_p = df.withColumn('Importe del Expediente',F.col('Importe del Expediente').cast('float')
        ).withColumn('CR_Porcentaje',F.col('CR_Porcentaje').cast('float'))

In [15]:
df_p.printSchema()

root
 |-- Nº Expediente: string (nullable = true)
 |-- Tramo: string (nullable = true)
 |-- País: string (nullable = true)
 |-- Clase Producto Financiero: string (nullable = true)
 |-- Divisa Base: string (nullable = true)
 |-- Importe del Expediente: float (nullable = true)
 |-- CR_Porcentaje: float (nullable = true)



In [20]:
df_p.show(5,False)

+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
|Nº Expediente|Tramo|País    |Clase Producto Financiero|Divisa Base|Importe del Expediente|CR_Porcentaje|
+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
|108870       |2    |PORTUGAL|Amortizing               |EUR        |28062.51              |4.0          |
|108949       |2    |FRANCIA |Amortizing               |EUR        |1335001.8             |5.6          |
|108949       |3    |FRANCIA |Amortizing               |EUR        |1304587.8             |5.6          |
|108949       |4    |FRANCIA |Amortizing               |EUR        |1326868.0             |5.6          |
|108949       |5    |FRANCIA |Amortizing               |EUR        |1618368.9             |5.6          |
+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
only showing top 5 rows



In [21]:
df_p.where(F.col('Importe del Expediente') >= 0).show(5,False)

+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
|Nº Expediente|Tramo|País    |Clase Producto Financiero|Divisa Base|Importe del Expediente|CR_Porcentaje|
+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
|108870       |2    |PORTUGAL|Amortizing               |EUR        |28062.51              |4.0          |
|108949       |2    |FRANCIA |Amortizing               |EUR        |1335001.8             |5.6          |
|108949       |3    |FRANCIA |Amortizing               |EUR        |1304587.8             |5.6          |
|108949       |4    |FRANCIA |Amortizing               |EUR        |1326868.0             |5.6          |
|108949       |5    |FRANCIA |Amortizing               |EUR        |1618368.9             |5.6          |
+-------------+-----+--------+-------------------------+-----------+----------------------+-------------+
only showing top 5 rows



In [38]:
import pandas as pd
import numpy as np

# Asegurándonos de que los datos están en el formato correcto
raw_data = df.toPandas() #pd.read_csv(path_csv)
raw_data.columns = ['Nº Expediente', 'Tramo', 'País', 'Clase Producto Financiero', 'Divisa Base', 'Importe del Expediente', 'CR_Porcentaje']


In [39]:
# raw_data.info(verbose = True, null_counts = False)

In [40]:
raw_data['Nº Expediente'] = raw_data['Nº Expediente'].astype('str') 
raw_data['Tramo'] = raw_data['Tramo'].astype('str') 
raw_data['País'] = raw_data['País'].astype('str') 
raw_data['Clase Producto Financiero'] = raw_data['Clase Producto Financiero'].astype('str') 
raw_data['Divisa Base'] = raw_data['Divisa Base'].astype('str') 
raw_data['Importe del Expediente'] = raw_data['Importe del Expediente'].astype(float)
raw_data['CR_Porcentaje'] = raw_data['CR_Porcentaje'].astype(float)

## Preparamos los datos

In [41]:
#Eliminamos registros con Importes nan y que sean cero.
data = raw_data.dropna(subset=['Importe del Expediente'])
data = data[data['Importe del Expediente'] >= 0]

# Ordenar los datos por porcentaje de riesgo descendente
df = data.sort_values(by='CR_Porcentaje', ascending=False).reset_index(drop=True)

# Convertir datos a listas para su uso en Pulp y OR-Tools
importes = data['Importe del Expediente'].tolist()
riesgos = data['CR_Porcentaje'].tolist()
divisas = data['Divisa Base'].tolist()
paises = data['País'].tolist()
ranking = data.index.values.tolist()
n = len(importes)


# Resolución sin programación lineal

In [42]:
import pandas as pd

# Inicializar las restricciones
total_importe = 0
divisa_importe = {divisa: 0 for divisa in df['Divisa Base'].unique()}
pais_importe = {pais: 0 for pais in df['País'].unique()}

# Lista para almacenar los índices de los expedientes seleccionados y sus proporciones
selected_expedients = []

# Agregar expedientes uno por uno
for i in df.index:
    expediente_importe = df.loc[i, 'Importe del Expediente']
    divisa = df.loc[i, 'Divisa Base']
    pais = df.loc[i, 'País']

    max_proportion = 1.0
    if total_importe + expediente_importe > 230000000:
        max_proportion = (230000000 - total_importe) / expediente_importe
    if divisa_importe[divisa] + expediente_importe > 75000000:
        max_proportion = min(max_proportion, (75000000 - divisa_importe[divisa]) / expediente_importe)
    if pais_importe[pais] + expediente_importe > 75000000:
        max_proportion = min(max_proportion, (75000000 - pais_importe[pais]) / expediente_importe)

    if max_proportion > 0:
        # Agregar el expediente y la proporción seleccionada
        selected_expedients.append((i, max_proportion))

        # Actualizar las restricciones
        total_importe += expediente_importe * max_proportion
        divisa_importe[divisa] += expediente_importe * max_proportion
        pais_importe[pais] += expediente_importe * max_proportion

        # Terminar si llegamos al límite total
        if total_importe >= 230000000:
            break

# Obtener los resultados
result_data = []
for i, proportion in selected_expedients:
    row = df.loc[i].copy()
    row['porcentaje optimo'] = proportion
    result_data.append(row)

result_df = pd.DataFrame(result_data)

result_df['importe optimo'] = result_df['porcentaje optimo'] * result_df['Importe del Expediente']

print(result_df['CR_Porcentaje'].sum())

result_df


149.99999999999997


,Nº Expediente,Tramo,País,Clase Producto Financiero,Divisa Base,Importe del Expediente,CR_Porcentaje,porcentaje optimo,importe optimo
0,273067,1,ALEMANIA,Amortizing,EUR,5.009869e+06,20.0,1.000000,5009868.87
1,762605,1,BELGICA,Amortizing,EUR,1.974614e+07,20.0,1.000000,19746139.99
2,789686,9,ESPAÑA,Amortizing,EUR,1.193516e+07,9.2,1.000000,11935157.58
3,789686,4,ESPAÑA,Amortizing,EUR,5.356645e+06,9.2,1.000000,5356644.81
4,789686,3,ESPAÑA,Amortizing,EUR,5.272117e+06,9.2,1.000000,5272116.64
5,789686,2,ESPAÑA,Amortizing,EUR,5.226640e+06,9.2,1.000000,5226639.96
6,789686,1,ESPAÑA,Amortizing,EUR,8.969408e+06,9.2,1.000000,8969407.68
7,789686,5,ESPAÑA,Amortizing,EUR,1.681453e+06,9.2,1.000000,1681452.76
8,789686,7,ESPAÑA,Amortizing,EUR,3.463644e+06,9.2,1.000000,3463644.37
9,789686,8,ESPAÑA,Amortizing,EUR,5.477639e+06,9.2,1.000000,5477639.14


In [43]:
grouped_data = result_df.groupby('Divisa Base')['importe optimo'].sum().reset_index()
print(grouped_data['importe optimo'].sum())
grouped_data

230000000.0


,Divisa Base,importe optimo
0,CAD,5000000.0
1,EUR,75000000.0
2,GBP,75000000.0
3,USD,75000000.0


In [44]:
grouped_data = result_df.groupby('País')['importe optimo'].sum().reset_index()
print(grouped_data['importe optimo'].sum())
grouped_data

230000000.0


,País,importe optimo
0,ALEMANIA,5009868.87
1,BELGICA,19746139.99
2,CANADA,5000000.00
3,ESPAÑA,50243991.14
4,ESTADOS UNIDOS DE AMERICA,75000000.00
5,REINO UNIDO,75000000.00
